### import

In [1]:
from selenium import webdriver
import pandas as pd 
from bs4 import BeautifulSoup
import time

### open the chrome web driver

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(PATH)

### create the list of titles to search

In [3]:
title_list = ['Analytics+Manager', 'Data+Scientist', 'Data+Engineer', 'Data+Analyst', 'Analytics+Manager', 'Business+Intelligence+Analyst']

### scrap jobs and save to csv file

In [4]:
for title_ in title_list:
    
    df = pd.DataFrame(columns=["Title","Location","Company","Salary","Sponsored","Description"])
    
    for i in range(0, 500, 50):

        driver.get(f'https:///www.indeed.com/jobs?q=title%3A%28{title_}%29&l=United+States&jt=fulltime&sort=date&limit=50&start='+str(i))
        driver.implicitly_wait(4)

        for job in driver.find_elements_by_class_name('result'):

            soup = BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')

            #title
            try:
                title = soup.find("a",class_="jobtitle").text.replace("\n","").strip()
            except:
                title = 'None'
                
            #location
            try:
                location = soup.find(class_="location").text
            except:
                location = 'None'

            #company
            try:
                company = soup.find(class_="company").text.replace("\n","").strip()
            except:
                company = 'None'

            #salary
            try:
                salary = soup.find(class_="salary").text.replace("\n","").strip()
            except:
                salary = 'None'

            # sponsore flag
            try:
                sponsored = soup.find(class_="sponsoredGray").text
                sponsored = "Sponsored"
            except:
                sponsored = "Organic"


            # click the job card to extract description
            sum_div = job.find_element_by_xpath('./div[2]')
            try:
                sum_div.click()
            except:
                close_button = driver.find_elements_by_class_name('popover-x-button-close')[0]
                close_button.click()
                sum_div.click()

            ## Give time for iframe to load
            time.sleep(7.5)

            try:
                ## You have to switch to the iframe like so: ##
                driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
                
                try:
                    ## Insert text via xpath ## //*[@id="jobDescriptionText"]
                    job_desc = driver.find_element_by_xpath('//*[@id="jobDescriptionText"]').text
                except:
                    job_desc = 'None'
                
                ## Switch back to the "default content" (that is, out of the iframes) ##
                driver.switch_to.default_content()
            except:
                job_desc = 'None'

            # add to data frame
            df = df.append({'Title':title,'Location':location,"Company":company,"Salary":salary,
                            "Sponsored":sponsored,"Description":job_desc},ignore_index=True)

    # save to csv file
    df.to_csv(f'{title_}_US.csv',index=False)
    del df

### Run the same script for Canada jobs by changing the url to CA domain

### close the webdriver

In [5]:
driver.quit()